In [6]:
import pandas as pd
import warnings
from datetime import timedelta
import numpy as np
import datetime
warnings.filterwarnings("ignore")
from datetime import datetime

In [7]:
week = pd.read_csv("weekly_data_1.csv")
daily = pd.read_csv("daily_data.csv")

def tr(data):
    data['previous_close'] = data['close'].shift(1)
    data['high-low'] = abs(data['high'] - data['low'])
    data['high-pc'] = abs(data['high'] - data['previous_close'])
    data['low-pc'] = abs(data['low'] - data['previous_close'])

    tr = data[['high-low', 'high-pc', 'low-pc']].max(axis=1)

    return tr

def atr(df, period):
    df['tr'] = tr(df)
    atr = df['tr'].rolling(period).mean()
    return atr

def supertrend(df, period=10, atr_multiplier=3):
    hl2 = (df['high'] + df['low']) / 2
    df['atr'] = atr(df, period)
    df['upperband'] = hl2 + (atr_multiplier * df['atr'])
    df['lowerband'] = hl2 - (atr_multiplier * df['atr'])
    df['in_uptrend'] = True

    for current in range(1, len(df.index)):
        previous = current - 1

        if df['close'][current] > df['upperband'][previous]:
            df['in_uptrend'][current] = True
        elif df['close'][current] < df['lowerband'][previous]:
            df['in_uptrend'][current] = False
        else:
            df['in_uptrend'][current] = df['in_uptrend'][previous]

            if df['in_uptrend'][current] and df['lowerband'][current] < df['lowerband'][previous]:
                df['lowerband'][current] = df['lowerband'][previous]

            if not df['in_uptrend'][current] and df['upperband'][current] > df['upperband'][previous]:
                df['upperband'][current] = df['upperband'][previous]
        
    return df

week = supertrend(week)
#week['date'] = week['date'].astype('datetime64[ns]')
#week['date'] = week['date'].dt.date


week.to_csv("week_finally.csv", index = False)

In [8]:
# df = pd.read_csv("week_date.csv")

In [14]:
daily = supertrend(daily)

In [9]:
# df

In [13]:
week.to_csv("week_finally_1.csv", index = False)

In [10]:
week['date'][1250:1251]

1250    2020-02-10 00:00:00+05:30
Name: date, dtype: object

In [15]:
b = []
a = list(daily['date'])
for i in a:
    i = i[0:10]
    i = datetime.strptime(i, "%Y-%m-%d").date()
    b.append(i)

In [16]:
daily['date_'] = b

In [17]:
daily.to_csv("daily_finally_1.csv", index = False)

In [57]:
a = "2020-02-10 00:00:00+05:30"
#a = a[]

In [62]:
a[0:10]

'2020-02-10'

In [26]:
from dateutil import parser
s= '25 April, 2020, 2:50, pm, IST'
b = parser.parse(a)

In [30]:
b.year, b.month, b.day

(2020, 2, 10)

In [36]:
from datetime import datetime

In [50]:
a = datetime.strptime("2020-02-10", "%Y-%m-%d")

In [54]:
str(a.date())

'2020-02-10'

In [52]:
a

datetime.datetime(2020, 2, 10, 0, 0)

In [39]:
week['date'] =  pd.to_datetime(week['date'], format='%y-%m-%d %Z')

ValueError: time data '1996-02-12 00:00:00+05:30' does not match format '%y-%m-%d %Z' (match)

In [2]:


trend_set = []
in_uptrend = list(week['in_uptrend'])
date = list(week['date'])


val = 0
#_status = []
for i in range(len(in_uptrend)):
    if i == val:
        #print(date[i])
        trend_set.append(date[i])
        status = in_uptrend[i]
        #_status.append(status)
    if status != in_uptrend[i]:
        trend_set.append(date[i])
        status = in_uptrend[i]
        #_status.append(status)
        #print(date[i])
        
dict_ = {}
for i in range(0, len(trend_set),2):
    try:
        dict_[trend_set[i]] = week.iloc[week.loc[week['date'] == datetime.date(trend_set[i+1].year, trend_set[i+1].month, trend_set[i+1].day)].index[0] - 1]['date']
    except:
        dict_[trend_set[i]] = np.nan

In [4]:
dict_

{datetime.date(1996, 2, 11): datetime.date(1996, 9, 1),
 datetime.date(1997, 1, 5): datetime.date(1997, 10, 19),
 datetime.date(1998, 3, 22): datetime.date(1998, 5, 17),
 datetime.date(1999, 1, 3): datetime.date(2000, 4, 9),
 datetime.date(2001, 2, 4): datetime.date(2001, 3, 4),
 datetime.date(2001, 11, 11): datetime.date(2002, 5, 19),
 datetime.date(2002, 11, 24): datetime.date(2003, 3, 30),
 datetime.date(2003, 6, 15): datetime.date(2004, 5, 2),
 datetime.date(2004, 9, 26): datetime.date(2005, 4, 17),
 datetime.date(2005, 6, 19): datetime.date(2006, 5, 14),
 datetime.date(2006, 10, 8): datetime.date(2007, 2, 18),
 datetime.date(2007, 5, 13): datetime.date(2007, 8, 5),
 datetime.date(2007, 9, 16): datetime.date(2008, 1, 27),
 datetime.date(2009, 4, 5): datetime.date(2011, 1, 16),
 datetime.date(2012, 2, 12): datetime.date(2012, 5, 20),
 datetime.date(2012, 9, 9): datetime.date(2013, 3, 10),
 datetime.date(2013, 5, 5): datetime.date(2013, 8, 4),
 datetime.date(2013, 10, 6): datetime.da

In [ ]:
daily = supertrend(daily)
daily['date'] = daily['date'].astype('datetime64[ns]')
daily['date'] = daily['date'].dt.date
#daily.to_csv("daily___.csv", index = False)

daily_index_range = {}
for k,v in dict_.items():
    try:
        daily_index_range[daily.loc[daily['date'] == datetime.date(k.year, k.month, k.day)].index[0]] = daily.loc[daily['date'] == datetime.date(v.year, v.month, v.day)].index[0]
    except:
        try:
            v =  v - timedelta(days=1)
            daily_index_range[daily.loc[daily['date'] == datetime.date(k.year, k.month, k.day)].index[0]] = daily.loc[daily['date'] == datetime.date(v.year, v.month, v.day)].index[0]
        except:
            try:
                k =  k - timedelta(days=1)
                v =  v - timedelta(days=2)
                daily_index_range[daily.loc[daily['date'] == datetime.date(k.year, k.month, k.day)].index[0]] = daily.loc[daily['date'] == datetime.date(v.year, v.month, v.day)].index[0]
            except:
                k =  k - timedelta(days=2)
                daily_index_range[daily.loc[daily['date'] == datetime.date(k.year, k.month, k.day)].index[0]] = np.nan
                #print(k,v)

In [3]:
c = 1
for k,v in daily_index_range.items():
    if (c%2!=0):
        #print(k,v, "True")
        df = pd.DataFrame()
        date = daily['date'][k:v]
        close = daily['close'][k:v]
        upperband = daily['upperband'][k:v]
        in_uptrend = daily['in_uptrend'][k:v]
        #status = "True"
        
        df['date'] = date
        df['close'] = close
        df['upperband'] = upperband
        df['in_uptrend'] = in_uptrend
        df['status'] = True
        #df.to_excel("name")
        
        
        df = df.reset_index()
        del df['index']
        print(df)
#         in_uptrend = df['in_uptrend']
#         date = list(df['date'])
#         flag = df['status'][0]     
        
#         val = 0
#         trend_set = []
#         #dict_
#         for i in range(len(in_uptrend)):
#             if i==val:
#                 status = in_uptrend[i]
#                 trend_set.append(date[i])
#             if status != in_uptrend[i]:
#                 status = in_uptrend[i]
#                 trend_set.append(date[i])
                
#             dict_ = {}
#             date_entry = []
#             date_exit = []
#             price_entry = []
#             price_exit = []
#             order = []
#             for i in range(0, len(trend_set),2):
#                 if i > 0:
#                     try:
#                         dict_[trend_set[i]] = trend_set[i+1] 

#                         date_entry.append(list(daily.loc[daily['date'] == datetime.date(trend_set[i].year, trend_set[i].month, trend_set[i].day)]['date'])[0])
#                         date_exit.append(daily.iloc[daily.loc[daily['date'] == datetime.date(trend_set[i+1].year, trend_set[i+1].month, trend_set[i+1].day)].index[0] - 1]['date'])

#                         price_entry.append(list(daily.loc[daily['date'] == datetime.date(trend_set[i].year, trend_set[i].month, trend_set[i].day)]['close'])[0])
#                         price_exit.append(daily.iloc[daily.loc[daily['date'] == datetime.date(trend_set[i+1].year, trend_set[i+1].month, trend_set[i+1].day)].index[0] - 1]['close'])

#                         order.append("BUY")
#                         dict_[list(daily.loc[daily['date'] == datetime.date(trend_set[i].year, trend_set[i].month, trend_set[i].day)]['close'])[0]] = daily.iloc[daily.loc[daily['date'] == datetime.date(trend_set[i+1].year, trend_set[i+1].month, trend_set[i+1].day)].index[0] - 1]['close']
#                     except:
#                         pass          
                    
#             dff = pd.DataFrame()
#             dff['date_entry'] = date_entry
#             dff['date_exit'] = date_exit
#             dff['price_entry'] = price_entry
#             dff['price_exit'] = price_exit
#             dff['order'] = order
#             print(dff)
    else:
        print(k,v, "False")

        df = pd.DataFrame()
        date = daily['date'][k:v]
        close = daily['close'][k:v]
        upperband = daily['upperband'][k:v]
        in_uptrend = daily['in_uptrend'][k:v]
        #status = "True"

        df['date'] = date
        df['close'] = close
        df['upperband'] = upperband
        df['in_uptrend'] = in_uptrend
        df['status'] = True
        #df.to_excel("name")


        df = df.reset_index()
        del df['index']
        print(df)
#         in_uptrend = df['in_uptrend']
#         date = list(df['date'])
#         flag = df['status'][0]     

#         val = 0
#         trend_set = []
#         #dict_
#         for i in range(len(in_uptrend)):
#             if i==val:
#                 status = in_uptrend[i]
#                 trend_set.append(date[i])
#             if status != in_uptrend[i]:
#                 status = in_uptrend[i]
#                 trend_set.append(date[i])

#             dict_ = {}
#             date_entry = []
#             date_exit = []
#             price_entry = []
#             price_exit = []
#             order = []
#             for i in range(0, len(trend_set),2):
#                 if i > 0:
#                     try:
#                         dict_[trend_set[i]] = trend_set[i+1] 

#                         date_entry.append(list(daily.loc[daily['date'] == datetime.date(trend_set[i].year, trend_set[i].month, trend_set[i].day)]['date'])[0])
#                         date_exit.append(daily.iloc[daily.loc[daily['date'] == datetime.date(trend_set[i+1].year, trend_set[i+1].month, trend_set[i+1].day)].index[0] - 1]['date'])

#                         price_entry.append(list(daily.loc[daily['date'] == datetime.date(trend_set[i].year, trend_set[i].month, trend_set[i].day)]['close'])[0])
#                         price_exit.append(daily.iloc[daily.loc[daily['date'] == datetime.date(trend_set[i+1].year, trend_set[i+1].month, trend_set[i+1].day)].index[0] - 1]['close'])

#                         order.append("SELL")
#                         dict_[list(daily.loc[daily['date'] == datetime.date(trend_set[i].year, trend_set[i].month, trend_set[i].day)]['close'])[0]] = daily.iloc[daily.loc[daily['date'] == datetime.date(trend_set[i+1].year, trend_set[i+1].month, trend_set[i+1].day)].index[0] - 1]['close']
#                     except:
#                         pass          

#             dff = pd.DataFrame()
#             dff['date_entry'] = date_entry
#             dff['date_exit'] = date_exit
#             dff['price_entry'] = price_entry
#             dff['price_exit'] = price_exit
#             dff['order'] = order
#             print(dff)
        
    c+=1
    #break

           date    close  upperband  in_uptrend  status
0    1996-02-11  1018.51        NaN        True    True
1    1996-02-12  1027.35        NaN        True    True
2    1996-02-13  1051.99        NaN        True    True
3    1996-02-14  1046.24        NaN        True    True
4    1996-02-15  1023.08        NaN        True    True
..          ...      ...        ...         ...     ...
134  1996-08-25  1009.04   1018.756       False    True
135  1996-08-26  1014.68   1018.756       False    True
136  1996-08-27  1027.08   1068.714        True    True
137  1996-08-28  1025.85   1076.706        True    True
138  1996-08-29  1029.00   1069.694        True    True

[139 rows x 5 columns]
224 415 False
           date    close  upperband  in_uptrend  status
0    1997-01-05   927.80    990.408        True    True
1    1997-01-06   928.80    984.833        True    True
2    1997-01-07   946.70   1005.327        True    True
3    1997-01-08   956.60   1008.420        True    True
4    1997-

56  2020-01-15  12355.50  12720.150        True    True
6062 nan False


TypeError: cannot do slice indexing on RangeIndex with these indexers [nan] of type float

In [23]:
c = 1
for k,v in daily_index_range.items():
    if (c%2!=0):
        print(k,v, "True")
        df = pd.DataFrame()
        date = daily['date'][k:v+1]
        close = daily['close'][k:v+1]
        upperband = daily['upperband'][k:v+1]
        in_uptrend = daily['in_uptrend'][k:v+1]
        status = "True"
        df['date'] = date
        df['close'] = close
        df['upperband'] = upperband
        df['in_uptrend'] = in_uptrend
        df['status'] = True
        #df.to_excel("name")
        
    else:
        print(k,v, "False")
    c+=1
    break

1 140 True


In [54]:
val = 0
trend_set = []
dict_
for i in range(len(in_uptrend)):
    if i==val:
        status = in_uptrend[i]
        trend_set.append(date[i])
    if status != in_uptrend[i]:
        status = in_uptrend[i]
        trend_set.append(date[i])

In [93]:
dict_ = {}
date_entry = []
date_exit = []
price_entry = []
price_exit = []
order = []
for i in range(0, len(trend_set),2):
    if i >0:
        try:
            dict_[trend_set[i]] = trend_set[i+1] 
            
            date_entry.append(list(daily.loc[daily['date'] == datetime.date(trend_set[i].year, trend_set[i].month, trend_set[i].day)]['date'])[0])
            date_exit.append(daily.iloc[daily.loc[daily['date'] == datetime.date(trend_set[i+1].year, trend_set[i+1].month, trend_set[i+1].day)].index[0] - 1]['date'])
            
            price_entry.append(list(daily.loc[daily['date'] == datetime.date(trend_set[i].year, trend_set[i].month, trend_set[i].day)]['close'])[0])
            price_exit.append(daily.iloc[daily.loc[daily['date'] == datetime.date(trend_set[i+1].year, trend_set[i+1].month, trend_set[i+1].day)].index[0] - 1]['close'])
            
            order.append("BUY")
            dict_[list(daily.loc[daily['date'] == datetime.date(trend_set[i].year, trend_set[i].month, trend_set[i].day)]['close'])[0]] = daily.iloc[daily.loc[daily['date'] == datetime.date(trend_set[i+1].year, trend_set[i+1].month, trend_set[i+1].day)].index[0] - 1]['close']
        except:
            pass
        #dict_[trend_set[i]] = np.nan

In [94]:
dff = pd.DataFrame()
dff['date_entry'] = date_entry
dff['date_exit'] = date_exit
dff['price_entry'] = price_entry
dff['price_exit'] = price_exit
dff['order'] = order

In [95]:
dff

,date_entry,date_exit,price_entry,price_exit,order
0,1996-03-27,1996-05-08,997.32,1090.53,BUY
1,1996-06-02,1996-06-23,1108.86,1146.78,BUY


In [78]:
dict_

{1018.51: 988.62,
 997.32: 1090.53,
 1108.86: 1146.78,
 datetime.date(1996, 8, 27): nan}

In [ ]:
dict(itertools.islice(d.items(), 1,3))

In [83]:
import itertools

d = {1: 2, 3: 4, 5: 6}

dict(itertools.islice(d.items(), 1,3))

{3: 4, 5: 6}

In [56]:
df = df.reset_index()
del df['index']

in_uptrend = df['in_uptrend']
date = list(df['date'])
status = df['status'][0]

In [33]:
val = 0
trend_set = []

for i in range(len(in_uptrend)):
    if i==val:
        _status = in_uptrend[i]
        trend_set.append(date[i])
        #print(_status)
    if status != in_uptrend[i]:
        _status = in_uptrend[i]
        trend_set.append(date[i])
        print(_status)

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False


In [38]:
val = 0
trend_set = []

for i in range(len(in_uptrend)):
    if i==val:
        status = in_uptrend[i]
        trend_set.append(date[i])
        print(status)
    if status != in_uptrend[i]:
        status = in_uptrend[i]
        trend_set.append(date[i])
        print(status)
    #ex

True
False
True
False
True
False
True


In [35]:
val = 0
trend_set = []

for i in range(len(in_uptrend)):
    if i==val:
        _status = in_uptrend[i]
        if status == _status:
            trend_set.append(date[i])
            print(_status)
    if status != in_uptrend[i]:
        _status = in_uptrend[i]
        if status == _status:
            trend_set.append(date[i])
            print(_status)

True


In [36]:
df

,date,close,upperband,in_uptrend,status
0,1996-02-11,1018.51,NaN,True,True
1,1996-02-12,1027.35,NaN,True,True
2,1996-02-13,1051.99,NaN,True,True
3,1996-02-14,1046.24,NaN,True,True
4,1996-02-15,1023.08,NaN,True,True
...,...,...,...,...,...
135,1996-08-26,1014.68,1018.756,False,True
136,1996-08-27,1027.08,1068.714,True,True
137,1996-08-28,1025.85,1076.706,True,True
138,1996-08-29,1029.00,1069.694,True,True


In [37]:
for i in range(len(in_uptrend)):
    try:
        if i == val:
            trend_set.append(date[i])
            status = in_uptrend[i]
            print(status)
            #print(trend_set)
            #_status.append(status)
        if status != in_uptrend[i]:
            print(status)
            #print(trend_set)
            trend_set.append(date[i])
            status = in_uptrend[i]
    except:
        pass

True
True
False
True
False
True
False


In [19]:



for i in range(len(in_uptrend)):
    try:
        if i == val:
            trend_set.append(date[i])
            status = in_uptrend[i]
            print(status)
            #print(trend_set)
            #_status.append(status)
        if status != in_uptrend[i]:
            print(status)
            #print(trend_set)
            trend_set.append(date[i])
            status = in_uptrend[i]
    except:
        pass

True
True
False
True
False
True
False


In [4]:
in_uptrend = df['in_uptrend']

In [9]:
# df['in_uptrend'].value_counts()

In [10]:
date = list(df['date'])
val = 0
trend_set = []
#_status = []
for i in range(len(in_uptrend)):
    try:
        if i == val:
            trend_set.append(date[i])
            status = in_uptrend[i]
            print(status)
            #print(trend_set)
            #_status.append(status)
        if status != in_uptrend[i]:
            print(status)
            #print(trend_set)
            trend_set.append(date[i])
            status = in_uptrend[i]
    except:
        pass

True
True
False
True
False
True
False
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
Tr

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [7]:
trend_set

[datetime.date(1996, 2, 11),
 datetime.date(1996, 2, 12),
 datetime.date(1996, 3, 11),
 datetime.date(1996, 3, 28),
 datetime.date(1996, 5, 12),
 datetime.date(1996, 6, 3),
 datetime.date(1996, 6, 25),
 datetime.date(1996, 8, 28)]

In [8]:
date = daily['date']
close = daily['close']
upperband = daily['upperband']
in_uptrend = daily['in_uptrend']

In [26]:
len(daily_index_range)

24

In [27]:
len(_status)

47

In [28]:

        #print(k,v)
        #pass
        #daily_index_range[daily.loc[daily['date'] == datetime.date(k.year, k.month, k.day)].index[0]] = np.nan

2020-07-18 nan


,date,open,high,low,close,previous_close,high-low,high-pc,low-pc,tr,atr,upperband,lowerband,in_uptrend
5144,2016-10-29,8672.35,8678.25,8616.25,8625.7,8638.0,62.0,40.25,21.75,62.0,76.7,8823.29,8417.15,False


In [18]:
#daily
for k,v in dict_.items():
    try:
        print(daily.loc[daily['date'] == datetime.date(k.year, k.month, k.day)].index[0], daily.loc[daily['date'] == datetime.date(v.year, v.month, v.day)].index[0])
    except:
        print(k,v)

1 140
224 415
519 555
716 1036
1243 1263
1435 1563
1692 1779
1831 2054
2159 2296
2342 2564
2668 2756
2812 2872
2901 2993
3273 3712
3979 4046
4125 4248
4283 4349
4390 4765
2016-05-22 2016-10-30
5212 5474
5565 5612
5730 5819
2019-10-27 2020-01-19
2020-07-19 nan


In [23]:
trend_set[1] - timedelta(days=1)

datetime.date(1996, 9, 7)

In [5]:
daily

,date,open,high,low,close
0,1996-02-09 00:00:00+05:30,1001.85,1033.02,980.27,1033.02
1,1996-02-12 00:00:00+05:30,1032.47,1045.81,1018.51,1018.51
2,1996-02-13 00:00:00+05:30,1019.69,1030.30,1006.28,1027.35
3,1996-02-14 00:00:00+05:30,1040.91,1067.49,1040.91,1051.99
4,1996-02-15 00:00:00+05:30,1048.14,1053.19,1042.35,1046.24
...,...,...,...,...,...
6417,2021-12-20 00:00:00+05:30,16824.25,16840.10,16410.20,16614.20
6418,2021-12-21 00:00:00+05:30,16773.15,16936.40,16688.25,16770.85
6419,2021-12-22 00:00:00+05:30,16865.55,16971.00,16819.50,16955.45
6420,2021-12-23 00:00:00+05:30,17066.80,17118.65,17015.55,17072.60


1996-02-11
1996-09-08
1997-01-05
1997-10-26
1998-03-22
1998-05-24
1999-01-03
2000-04-16
2001-02-04
2001-03-11
2001-11-11
2002-05-26
2002-11-24
2003-04-06
2003-06-15
2004-05-09
2004-09-26
2005-04-24
2005-06-19
2006-05-21
2006-10-08
2007-02-25
2007-05-13
2007-08-12
2007-09-16
2008-02-03
2009-04-05
2011-01-23
2012-02-12
2012-05-27
2012-09-09
2013-03-17
2013-05-05
2013-08-11
2013-10-06
2015-04-26
2016-05-22
2016-11-06
2017-02-05
2018-03-04
2018-07-08
2018-09-23
2019-03-10
2019-07-28
2019-10-27
2020-01-26
2020-07-19
